In [12]:
pip install os
pip install glob
pip install pandas
pip install reduce
pip install ElementTree 

In [12]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et

In [13]:
xml_list = glob('./data_images/*.xml')
#data cleaning
xml_list = list(map(lambda x: x.replace('\\','/'),xml_list))

In [14]:
xml_list

[]

In [8]:
#read xml
#extract :
#filename, size(w,h), object(name,xmin,xmax,ymin,ymax)
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    
    #filename
    image_name = root.find('filename').text
    #width and height
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
    
    return parser

In [9]:
parser_all = list(map(extract_text,xml_list))

In [10]:
parser_all

[]

In [115]:
data = reduce(lambda x, y : x+y, parser_all)

In [116]:
df = pd.DataFrame(data,columns = ['filename','width', 'height','name','xmin', 'xmax', 'ymin','ymax'])

In [117]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax
0,bad1_mp4-0000_jpg.rf.4e19812278be2eb9c877d44ae...,640,640,shuttle-P15u,304,307,361,368
1,bad1_mp4-0001_jpg.rf.172fa7d0b38232151a6f0492d...,640,640,shuttle-P15u,304,307,360,367
2,bad1_mp4-0002_jpg.rf.dc95a107f082189fbf81c59a4...,640,640,shuttle-P15u,305,307,360,365
3,bad1_mp4-0003_jpg.rf.c3f72fd3fe4a0adde8a8134f3...,640,640,shuttle-P15u,305,307,358,363
4,bad1_mp4-0004_jpg.rf.29decd8ae789e10ce117c261d...,640,640,shuttle-P15u,305,308,357,362


In [118]:
df.shape

(93, 8)

In [119]:
df['name'].value_counts()

name
shuttle-P15u    93
Name: count, dtype: int64

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  93 non-null     object
 1   width     93 non-null     object
 2   height    93 non-null     object
 3   name      93 non-null     object
 4   xmin      93 non-null     object
 5   xmax      93 non-null     object
 6   ymin      93 non-null     object
 7   ymax      93 non-null     object
dtypes: object(8)
memory usage: 5.9+ KB


In [121]:
#type conversion
cols = ['width','height','xmin','xmax','ymin','ymax']
df[cols] = df[cols].astype(int)
df.info()

#center x, center y
df['center_x'] = ((df['xmax']+df['xmin'])/2)/df['width']
df['center_y'] = ((df['ymax']+df['ymin'])/2)/df['height']
#w
df['w'] = (df['xmax'] - df['xmin'])/df['width']
#h
df['h'] = (df['ymax'] - df['ymin'])/df['height']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  93 non-null     object
 1   width     93 non-null     int32 
 2   height    93 non-null     int32 
 3   name      93 non-null     object
 4   xmin      93 non-null     int32 
 5   xmax      93 non-null     int32 
 6   ymin      93 non-null     int32 
 7   ymax      93 non-null     int32 
dtypes: int32(6), object(2)
memory usage: 3.8+ KB


In [122]:
df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,bad1_mp4-0000_jpg.rf.4e19812278be2eb9c877d44ae...,640,640,shuttle-P15u,304,307,361,368,0.477344,0.569531,0.004687,0.010937
1,bad1_mp4-0001_jpg.rf.172fa7d0b38232151a6f0492d...,640,640,shuttle-P15u,304,307,360,367,0.477344,0.567969,0.004687,0.010937
2,bad1_mp4-0002_jpg.rf.dc95a107f082189fbf81c59a4...,640,640,shuttle-P15u,305,307,360,365,0.478125,0.566406,0.003125,0.007812
3,bad1_mp4-0003_jpg.rf.c3f72fd3fe4a0adde8a8134f3...,640,640,shuttle-P15u,305,307,358,363,0.478125,0.563281,0.003125,0.007812
4,bad1_mp4-0004_jpg.rf.29decd8ae789e10ce117c261d...,640,640,shuttle-P15u,305,308,357,362,0.478906,0.561719,0.004687,0.007812


In [130]:
img = df['filename'].unique()

In [132]:
len(img)

93

In [136]:
#80 train and 20% test
img_df = pd.DataFrame(img,columns=['filename'])
img_train = tuple(img_df.sample(frac=0.8)['filename']) #random 80%


In [143]:
img_test = tuple(img_df.query(f'filename not in {img_train}')['filename'])

In [146]:
len(img_train),len(img_test)

(74, 19)

In [148]:
train_df = df.query(f'filename in {img_train}')
test_df = df.query(f'filename in {img_test}')

In [150]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
1,bad1_mp4-0001_jpg.rf.172fa7d0b38232151a6f0492d...,640,640,shuttle-P15u,304,307,360,367,0.477344,0.567969,0.004687,0.010937
2,bad1_mp4-0002_jpg.rf.dc95a107f082189fbf81c59a4...,640,640,shuttle-P15u,305,307,360,365,0.478125,0.566406,0.003125,0.007812
3,bad1_mp4-0003_jpg.rf.c3f72fd3fe4a0adde8a8134f3...,640,640,shuttle-P15u,305,307,358,363,0.478125,0.563281,0.003125,0.007812
4,bad1_mp4-0004_jpg.rf.29decd8ae789e10ce117c261d...,640,640,shuttle-P15u,305,308,357,362,0.478906,0.561719,0.004687,0.007812
5,bad1_mp4-0005_jpg.rf.2585b87e769dd837bad724ea2...,640,640,shuttle-P15u,306,308,355,360,0.479687,0.558594,0.003125,0.007812


In [152]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h
0,bad1_mp4-0000_jpg.rf.4e19812278be2eb9c877d44ae...,640,640,shuttle-P15u,304,307,361,368,0.477344,0.569531,0.004687,0.010937
6,bad1_mp4-0009_jpg.rf.79747d4dc15cfa7b7b37a9d08...,640,640,shuttle-P15u,321,324,300,306,0.503906,0.473438,0.004687,0.009375
11,bad1_mp4-0014_jpg.rf.de5e7762b524554cbffeaa3ab...,640,640,shuttle-P15u,343,346,264,270,0.538281,0.417187,0.004687,0.009375
15,bad1_mp4-0018_jpg.rf.7d379e64cb25d968cd13adfa3...,640,640,shuttle-P15u,357,362,265,271,0.561719,0.418750,0.007812,0.009375
19,bad1_mp4-0023_jpg.rf.6a3941d3c31169be391e52794...,640,640,shuttle-P15u,373,377,239,246,0.585938,0.378906,0.006250,0.010937


In [158]:
def label_encoding(x):
    labels = {'shuttle-P15u': 0 }
    return labels[x]

In [161]:
train_df['id'] = train_df['name'].apply(label_encoding)
test_df['id'] = test_df['name'].apply(label_encoding)

C:\Users\sayam\AppData\Local\Temp\ipykernel_2376\3116627190.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['id'] = train_df['name'].apply(label_encoding)
C:\Users\sayam\AppData\Local\Temp\ipykernel_2376\3116627190.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['id'] = test_df['name'].apply(label_encoding)


In [166]:
train_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
1,bad1_mp4-0001_jpg.rf.172fa7d0b38232151a6f0492d...,640,640,shuttle-P15u,304,307,360,367,0.477344,0.567969,0.004687,0.010937,0
2,bad1_mp4-0002_jpg.rf.dc95a107f082189fbf81c59a4...,640,640,shuttle-P15u,305,307,360,365,0.478125,0.566406,0.003125,0.007812,0
3,bad1_mp4-0003_jpg.rf.c3f72fd3fe4a0adde8a8134f3...,640,640,shuttle-P15u,305,307,358,363,0.478125,0.563281,0.003125,0.007812,0
4,bad1_mp4-0004_jpg.rf.29decd8ae789e10ce117c261d...,640,640,shuttle-P15u,305,308,357,362,0.478906,0.561719,0.004687,0.007812,0
5,bad1_mp4-0005_jpg.rf.2585b87e769dd837bad724ea2...,640,640,shuttle-P15u,306,308,355,360,0.479687,0.558594,0.003125,0.007812,0


In [165]:
test_df.head()

,filename,width,height,name,xmin,xmax,ymin,ymax,center_x,center_y,w,h,id
0,bad1_mp4-0000_jpg.rf.4e19812278be2eb9c877d44ae...,640,640,shuttle-P15u,304,307,361,368,0.477344,0.569531,0.004687,0.010937,0
6,bad1_mp4-0009_jpg.rf.79747d4dc15cfa7b7b37a9d08...,640,640,shuttle-P15u,321,324,300,306,0.503906,0.473438,0.004687,0.009375,0
11,bad1_mp4-0014_jpg.rf.de5e7762b524554cbffeaa3ab...,640,640,shuttle-P15u,343,346,264,270,0.538281,0.417187,0.004687,0.009375,0
15,bad1_mp4-0018_jpg.rf.7d379e64cb25d968cd13adfa3...,640,640,shuttle-P15u,357,362,265,271,0.561719,0.418750,0.007812,0.009375,0
19,bad1_mp4-0023_jpg.rf.6a3941d3c31169be391e52794...,640,640,shuttle-P15u,373,377,239,246,0.585938,0.378906,0.006250,0.010937,0


In [168]:
#save image
import os
from shutil import move


In [171]:
train_folder = 'data_images/train'
test_folder = 'data_images/test'

os.mkdir(train_folder)
os.mkdir(test_folder)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data_images/train'

In [175]:
cols = ['filename','id','center_x','center_y','w','h']
groupby_obj_train = train_df[cols].groupby('filename')
groupby_obj_test = test_df[cols].groupby('filename')

In [186]:
#groupby_obj_train.get_group('bad1_mp4-0001_jpg.rf.172fa7d0b38232151a6f0492dda827ac.jpg').set_index('filename').to_csv('sample.txt',index = False,header=False)
# save each image in train/test folder in .txt
def save_data(filename,folder_path,group_obj):
    #move image
    src = os.path.join('data_images',filename)
    dst = os.path.join(folder_path,filename)
    move(src,dst) # move image to dst folder

    # save the labels
    text_filename = os.path.join(folder_path,
                                 os.path.splitext(filename)[0]+'.txt')
    group_obj.get_group(filename).set_index('filename').to_csv(text_filename,sep=' ',index = False,header=False)

In [190]:
filename_series = pd.Series(groupby_obj_train.groups.keys())

In [195]:
filename_series.apply(save_data,args=(train_folder,groupby_obj_train))

FileNotFoundError: [Errno 2] No such file or directory: 'data_images\\bad1_mp4-0001_jpg.rf.172fa7d0b38232151a6f0492dda827ac.jpg'

In [196]:
filename_series_test = pd.Series(groupby_obj_test.groups.keys())
filename_series_test.apply(save_data,args=(test_folder,groupby_obj_test))

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
dtype: object